Importing all necesarry Libraries

In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
!pip install folium
import folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


### 2. Data Extraction and Cleaning
We retreive the URL from wikipedia and create a Beautiful soup object. Then find the table and table data.
 List of Postal Codes of Given Wikipedia Page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)

In [5]:
data = response.text
soup = BeautifulSoup(data,'html.parser')
wiki_table=soup.find('table')
df = pd.read_html(str(wiki_table))[0]

In [6]:
#we create a dictionary having 3 keys PostalCode, Borough and Neighborhood.Exclude 'Not assigned'cells.
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [7]:
df.shape

(103, 3)

Upload a geospatial data in dataframe.Rename column 'Postal Code' as 'PostalCode' as it named in dataframe from part1

In [8]:
url='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_geospat=pd.read_csv(url)
df_geospat = df_geospat.rename(columns = {'Postal Code':'PostalCode'}) 

In [9]:
#Combine 2 dataframes on 'PostalCode' column
df1 = pd.merge(df, df_geospat, on = 'PostalCode')
df1.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


We work with only boroughs that contain the word 'DowntownToronto' in its name.

In [10]:
df2=df1[df1['Borough'].str.contains('Downtown Toronto')]
df2=df2.reset_index(drop=True)
df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
5,M6G,Downtown Toronto,Christie,43.669542,-79.422564
6,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
7,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
8,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


In [11]:
#create map of Toronto
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [12]:
 Toronto_map = folium.Map(location=[43.6534817, -79.3839347], zoom_start=14)
 Toronto_map 

Create map of neighborhoods in downtown Toronto

In [13]:
neighborhoods = folium.map.FeatureGroup()


for lat, lng, in zip(df2.Latitude, df2.Longitude):
    neighborhoods.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add neighborhoods to map
Toronto_map.add_child(neighborhoods)

Use Foursqare API to obltain list of venues in neighborhoods of Downtown Toronto

In [14]:
CLIENT_ID = 'JQ5MTMI2KD4L5A0ZMGPSNM5IFLLKJXMLW3NWKZHMY2BLDDCY' # your Foursquare ID
CLIENT_SECRET = 'VDOGQGZHGLNICON2W0INR1ITG32N2ZR2WFKWMTXGFD3HYWSD' # your Foursquare Secret
ACCESS_TOKEN = 'JBZIHJ2Q4NKN2XC2VRCNGCSLN0XRLLKHQVBEANML3UWM1OIY' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JQ5MTMI2KD4L5A0ZMGPSNM5IFLLKJXMLW3NWKZHMY2BLDDCY
CLIENT_SECRET:VDOGQGZHGLNICON2W0INR1ITG32N2ZR2WFKWMTXGFD3HYWSD


In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
toronto_downtown_venues = getNearbyVenues(names=df2['Neighborhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )


Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [17]:
toronto_downtown_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [18]:
toronto_downtown_venues.shape

(485, 7)

In [19]:
# Top 10 Categories
a=pd.Series(toronto_downtown_venues['Venue Category'])
a.value_counts()[:40]

Café                             47
Coffee Shop                      37
Restaurant                       17
Park                             15
Bakery                           14
Japanese Restaurant              13
Gastropub                        11
Hotel                            11
Seafood Restaurant               10
Art Gallery                       8
Beer Bar                          8
Farmers Market                    8
Italian Restaurant                7
American Restaurant               6
Pub                               6
Sandwich Place                    6
Dessert Shop                      6
Thai Restaurant                   6
Cocktail Bar                      6
Pizza Place                       6
Vegetarian / Vegan Restaurant     6
Gym / Fitness Center              5
Cheese Shop                       5
Creperie                          5
Theater                           5
Comfort Food Restaurant           5
Concert Hall                      5
Steakhouse                  

In [20]:
print('There are {} Uniques Categories.'.format(len(toronto_downtown_venues['Venue Category'].unique())))
toronto_downtown_venues.groupby('Neighborhood').count().head()

There are 147 Uniques Categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,30,30,30,30,30,30
Christie,15,15,15,15,15,15
Church and Wellesley,30,30,30,30,30,30


One Hot Encoding of Features

In [21]:
# one hot encoding
toronto_downtown_onehot = pd.get_dummies(toronto_downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood clumn back to dataframe
toronto_downtown_onehot['Neighborhood'] = toronto_downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_downtown_onehot.columns[-1]] + list(toronto_downtown_onehot.columns[:-1])
toronto_downtown_onehot = toronto_downtown_onehot[fixed_columns]
toronto_downtown_grouped = toronto_downtown_onehot.groupby('Neighborhood').mean().reset_index()
toronto_downtown_onehot.head(5)

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
num_top_venues = 5
for hood in toronto_downtown_grouped['Neighborhood']:
    print("---- "+hood+" ----")
    temp =toronto_downtown_grouped[toronto_downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Berczy Park ----
                venue  freq
0  Seafood Restaurant  0.07
1         Cheese Shop  0.07
2      Farmers Market  0.07
3        Cocktail Bar  0.07
4         Coffee Shop  0.03


---- CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport ----
                 venue  freq
0       Airport Lounge  0.12
1      Airport Service  0.12
2     Airport Terminal  0.12
3  Rental Car Location  0.06
4      Harbor / Marina  0.06


---- Central Bay Street ----
                venue  freq
0         Coffee Shop  0.23
1                Café  0.10
2  Italian Restaurant  0.07
3         Yoga Studio  0.03
4    Department Store  0.03


---- Christie ----
           venue  freq
0  Grocery Store  0.27
1           Café  0.20
2           Park  0.13
3    Coffee Shop  0.07
4      Nightclub  0.07


---- Church and Wellesley ----
                venue  freq
0    Sushi Restaurant  0.07
1         Coffee Shop  0.03
2      Breakfast Spot  0.03
3        Burge

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_downtown_grouped['Neighborhood']

for ind in np.arange(toronto_downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Farmers Market,Cheese Shop,Cocktail Bar,Fountain,Jazz Club,Coffee Shop,Park,French Restaurant,Museum
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Sculpture Garden,Rental Car Location,Plane,Boat or Ferry
2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Yoga Studio,Modern European Restaurant,Bubble Tea Shop,Portuguese Restaurant,Poke Place,Park,Middle Eastern Restaurant
3,Christie,Grocery Store,Café,Park,Coffee Shop,Candy Store,Nightclub,Baby Store,Italian Restaurant,Restaurant,Comic Shop
4,Church and Wellesley,Sushi Restaurant,Pizza Place,Breakfast Spot,Pub,Ramen Restaurant,Bookstore,Burger Joint,Restaurant,Juice Bar,Salon / Barbershop
5,"Commerce Court, Victoria Hotel",Café,Hotel,Gastropub,Coffee Shop,Restaurant,American Restaurant,Japanese Restaurant,Museum,Pub,Deli / Bodega
6,Enclave of M5E,Seafood Restaurant,Farmers Market,Cheese Shop,Café,Creperie,Beer Bar,Cocktail Bar,Museum,Food Truck,Park
7,"First Canadian Place, Underground city",Café,Restaurant,Coffee Shop,Seafood Restaurant,Steakhouse,Gym / Fitness Center,Concert Hall,Japanese Restaurant,Sandwich Place,Bakery
8,"Garden District, Ryerson",Café,Theater,Burrito Place,Plaza,Sandwich Place,Japanese Restaurant,Comic Shop,Ramen Restaurant,College Rec Center,Burger Joint
9,"Harbourfront East, Union Station, Toronto Islands",Park,Plaza,Café,Hotel,New American Restaurant,IT Services,Lake,Bistro,Bubble Tea Shop,Roof Deck


Clustering of neighborhoods

In [25]:
# Using K-Means to cluster neighborhood into 6 clusters
from sklearn.cluster import KMeans
toronto_downtown_grouped_clustering = toronto_downtown_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=6, random_state=0).fit(toronto_downtown_grouped_clustering)
kmeans.labels_

array([0, 5, 2, 4, 0, 3, 0, 3, 3, 0, 0, 2, 3, 1, 0, 0, 3, 3], dtype=int32)

In [26]:
neighborhoods_venues_sorted.insert(0,'cluster', kmeans.labels_)

toronto_downtown_merged =df2.iloc[:17,:]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_downtown_merged = toronto_downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_downtown_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Distribution Center,Performing Arts Venue,Pub,Restaurant,Mexican Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Café,Theater,Burrito Place,Plaza,Sandwich Place,Japanese Restaurant,Comic Shop,Ramen Restaurant,College Rec Center,Burger Joint
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Gastropub,Café,Farmers Market,Coffee Shop,Food Truck,Italian Restaurant,Park,Church,New American Restaurant,Middle Eastern Restaurant
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Seafood Restaurant,Farmers Market,Cheese Shop,Cocktail Bar,Fountain,Jazz Club,Coffee Shop,Park,French Restaurant,Museum
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Café,Italian Restaurant,Yoga Studio,Modern European Restaurant,Bubble Tea Shop,Portuguese Restaurant,Poke Place,Park,Middle Eastern Restaurant
5,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4,Grocery Store,Café,Park,Coffee Shop,Candy Store,Nightclub,Baby Store,Italian Restaurant,Restaurant,Comic Shop
6,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,3,Coffee Shop,Café,Seafood Restaurant,Steakhouse,Sushi Restaurant,Hotel,Lounge,Smoke Shop,Asian Restaurant,Speakeasy
7,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,0,Park,Plaza,Café,Hotel,New American Restaurant,IT Services,Lake,Bistro,Bubble Tea Shop,Roof Deck
8,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,3,Café,Coffee Shop,Restaurant,Japanese Restaurant,Pizza Place,Hotel,Museum,Concert Hall,Pub,Deli / Bodega
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,3,Café,Hotel,Gastropub,Coffee Shop,Restaurant,American Restaurant,Japanese Restaurant,Museum,Pub,Deli / Bodega


Explore sport venues

In [27]:
df3=toronto_downtown_venues 

In [28]:
df_fitness=df3.loc[df3['Venue Category']=='Gym / Fitness Center']
df_fitness

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
7,"Regent Park, Harbourfront",43.654260,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center
177,"Richmond, Adelaide, King",43.650571,-79.384568,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center
233,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center
259,"Commerce Court, Victoria Hotel",43.648198,-79.379817,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center
427,"First Canadian Place, Underground city",43.648429,-79.382280,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center


In [29]:
df_yoga=df3.loc[df3['Venue Category']=='Yoga Studio']
df_yoga

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
24,"Regent Park, Harbourfront",43.654260,-79.360636,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
143,Central Bay Street,43.657952,-79.387383,The Yoga Sanctuary,43.661499,-79.383636,Yoga Studio
290,"University of Toronto, Harbord",43.662696,-79.400049,Sivananda Yoga Centre,43.662754,-79.402951,Yoga Studio


In [30]:
df_gym=df3.loc[df3['Venue Category']=='Gym']
df_gym

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
68,St. James Town,43.651494,-79.375418,GoodLife Fitness Toronto 137 Yonge Street,43.651242,-79.378068,Gym
226,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,Equinox Bay Street,43.648100,-79.379989,Gym
257,"Commerce Court, Victoria Hotel",43.648198,-79.379817,Equinox Bay Street,43.648100,-79.379989,Gym
426,"First Canadian Place, Underground city",43.648429,-79.382280,Equinox Bay Street,43.648100,-79.379989,Gym


In [31]:
df_sport=pd.concat([df_yoga,df_gym,df_fitness],axis=0)
df_sport

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
24,"Regent Park, Harbourfront",43.654260,-79.360636,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
143,Central Bay Street,43.657952,-79.387383,The Yoga Sanctuary,43.661499,-79.383636,Yoga Studio
290,"University of Toronto, Harbord",43.662696,-79.400049,Sivananda Yoga Centre,43.662754,-79.402951,Yoga Studio
68,St. James Town,43.651494,-79.375418,GoodLife Fitness Toronto 137 Yonge Street,43.651242,-79.378068,Gym
226,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,Equinox Bay Street,43.648100,-79.379989,Gym
257,"Commerce Court, Victoria Hotel",43.648198,-79.379817,Equinox Bay Street,43.648100,-79.379989,Gym
426,"First Canadian Place, Underground city",43.648429,-79.382280,Equinox Bay Street,43.648100,-79.379989,Gym
7,"Regent Park, Harbourfront",43.654260,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center
177,"Richmond, Adelaide, King",43.650571,-79.384568,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center
233,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center


In [32]:
sport_venues = folium.map.FeatureGroup()


for lat, lng, in zip(df2.Latitude, df2.Longitude):
    sport_venues.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add neighborhoods to map
Toronto_map.add_child(sport_venues)

In [33]:

df4=df3[ (df3['Venue Category']=='Gym')| (df3['Venue Category']=='Gym / Fitness Center') | (df3['Venue Category']=='Yoga Studio')]
count_sport=df4['Venue Category'].value_counts().to_frame()
count_sport

,Venue Category
Gym / Fitness Center,5
Gym,4
Yoga Studio,3


In [34]:
df_veg=df3.loc[df3['Venue Category']=='Vegetarian / Vegan Restaurant']
df_veg

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
92,Berczy Park,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
137,Central Bay Street,43.657952,-79.387383,Vegetarian Haven,43.656016,-79.392758,Vegetarian / Vegan Restaurant
167,"Richmond, Adelaide, King",43.650571,-79.384568,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
340,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,Hibiscus,43.655454,-79.402439,Vegetarian / Vegan Restaurant
365,Enclave of M5E,43.646435,-79.374846,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
444,"First Canadian Place, Underground city",43.648429,-79.382280,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant


In [35]:
df_veg1=df_veg[['Neighborhood','Venue Category']]
df_veg1

,Neighborhood,Venue Category
92,Berczy Park,Vegetarian / Vegan Restaurant
137,Central Bay Street,Vegetarian / Vegan Restaurant
167,"Richmond, Adelaide, King",Vegetarian / Vegan Restaurant
340,"Kensington Market, Chinatown, Grange Park",Vegetarian / Vegan Restaurant
365,Enclave of M5E,Vegetarian / Vegan Restaurant
444,"First Canadian Place, Underground city",Vegetarian / Vegan Restaurant


In [36]:
df_sport1=df_sport[['Neighborhood','Venue Category']]
df_sport1

,Neighborhood,Venue Category
24,"Regent Park, Harbourfront",Yoga Studio
143,Central Bay Street,Yoga Studio
290,"University of Toronto, Harbord",Yoga Studio
68,St. James Town,Gym
226,"Toronto Dominion Centre, Design Exchange",Gym
257,"Commerce Court, Victoria Hotel",Gym
426,"First Canadian Place, Underground city",Gym
7,"Regent Park, Harbourfront",Gym / Fitness Center
177,"Richmond, Adelaide, King",Gym / Fitness Center
233,"Toronto Dominion Centre, Design Exchange",Gym / Fitness Center


Find neighborhoods where sport venues already located near vegeterian restaurants

In [37]:

df6 = pd.merge(df_sport1, df_veg1, on = 'Neighborhood')
df6.head()

,Neighborhood,Venue Category_x,Venue Category_y
0,Central Bay Street,Yoga Studio,Vegetarian / Vegan Restaurant
1,"First Canadian Place, Underground city",Gym,Vegetarian / Vegan Restaurant
2,"First Canadian Place, Underground city",Gym / Fitness Center,Vegetarian / Vegan Restaurant
3,"Richmond, Adelaide, King",Gym / Fitness Center,Vegetarian / Vegan Restaurant
